In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

In [ ]:
# MNIST dataset parameters.
num_classes=10 # 0 to 9 digits
num_features=784 # 28*28

learning_rate=0.01
training_steps=1000
batch_size=256
display_step=50

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test)=mnist.load_data()
x_train, x_test=np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test=x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test=x_train/255., x_test/255.

In [ ]:
train_data=tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
tf.one_hot(1, 10)

In [ ]:
W=tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b=tf.Variable(tf.zeros([num_classes]), name="bias")

def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W)+b)

def cross_entropy(y_pred, y_true):
    y_true=tf.one_hot(y_true, depth=num_classes)
    y_pred=tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred), 1))

def accuracy(y_pred, y_true):
    correct_prediction=tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer=tf.optimizers.SGD(learning_rate)

In [ ]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred=logistic_regression(x)
        loss=cross_entropy(pred, y)
        
    gradients=g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [ ]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    
    if step%display_step==0:
        pred=logistic_regression(batch_x)
        loss=cross_entropy(pred, batch_y)
        acc=accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

In [ ]:
pred=logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
n_images=5
test_images=x_test[:n_images]
predictions=logistic_regression(test_images)

for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction: %i" % np.argmax(predictions.numpy()[i]))